In [37]:
import numpy as np
import pandas as pd
import sklearn
import ot
import matplotlib.pyplot as plt



In [38]:
def compute_rmse(predictions, test_matrix):
  masked = np.ma.array(test_matrix, mask=np.isnan(test_matrix))
  diff = np.ma.subtract(predictions, masked)
  squared = np.ma.power(diff, 2)
  return np.ma.sqrt(np.ma.mean(squared))

In [39]:
ratings_train = np.load('../datasets/ratings_train.npy')
ratings_test = np.load('../datasets/ratings_test.npy')
m,n = ratings_train.shape
data = np.nan_to_num(ratings_train)
data_test = np.nan_to_num(ratings_test)


In [40]:
from sklearn.decomposition import NMF
import scipy.sparse as sps

csr_ratings_train = sps.csr_matrix(np.nan_to_num(ratings_train), shape=(610, 4980))
csr_ratings_train.eliminate_zeros()

nmf = NMF(n_components=10, init='nndsvd', random_state=0, max_iter=1000)
W = nmf.fit_transform(csr_ratings_train)
H = nmf.components_
flattened_array = np.unique(H.flatten())
sorted_unique = np.sort(flattened_array)
second_min = sorted_unique[1]
threshold = second_min/2
H[H < threshold] = threshold
flattened_array = np.unique(W.flatten())
sorted_unique = np.sort(flattened_array)
second_min = sorted_unique[1]
threshold = second_min/2
W[W < threshold] = threshold


In [41]:
print(H.shape)

(10, 4980)


In [43]:
from Matcher import Matcher, model_parameters,train_parameters

train_pi_sample = data / data.sum()
test_pi_sample = data_test/data_test.sum()
# RiOT
model = Matcher(train_pi_sample, W.T, H, r=5)
lam = 1
model_param = model_parameters(A0=np.eye(10, 10),
                                        gamma=0.2,
                                        const=1,
                                        degree=2,
                                        lam=lam,
                                        lambda_mu=1,
                                        lambda_nu=1,
                                        delta=0.005)
train_param = train_parameters(max_outer_iteration=20,
                                        max_inner_iteration=10,
                                        learning_rate=0.01)
model.riot(model_param, train_param)
test_r, test_c = train_pi_sample.sum(axis=1), train_pi_sample.sum(axis=0)
pred_pi = ot.rot(model.C, test_r, test_c, lam)[0]

print(compute_rmse(pred_pi,test_pi_sample))

outer iteration 0
inner iteration 0
inner iteration 0
inner iteration 0


KeyboardInterrupt: 